In [2]:
import pyscf
from pyscf import gto, scf, fci
import numpy as np

In [3]:
import numpy as np

# Distância de ligação C-H experimental (em Å)
r_ch = 1.087

# Coordenadas tetraédricas (baseadas em r_ch)
# A = r_ch / sqrt(3)
a = r_ch / np.sqrt(3)

mol = gto.M(
    atom=[
        ['C', ( 0.0,  0.0,  0.0)],
        ['H', (  a,    a,    a )],
        ['H', (  a,   -a,   -a )],
        ['H', ( -a,    a,   -a )],
        ['H', ( -a,   -a,    a )]
    ],
    basis='6-31g(d)',  # Base mínima
    spin=0,          # Singlete (10 elétrons)
    symmetry=True    # Habilitar simetria (Td)
)
mol.build()

In [4]:
# 1. Calcular a energia Hartree-Fock
myhf = scf.RHF(mol).run()

print(f"Energia Hartree-Fock (RHF): {myhf.e_tot:.8f} Hartrees")

converged SCF energy = -40.1948109553752
Energia Hartree-Fock (RHF): -40.19481096 Hartrees


In [5]:
# 2. Calcular a energia Full CI
# Inicializamos o resolvedor FCI com o objeto 'myhf' (que contém os orbitais)
myfci = fci.FCI(myhf)

myfci.nthreads = 40

# myfci.kernel() diagonaliza a matriz Hamiltoniana FCI
# Retorna: (energia_fci, vetor_de_onda_fci)
e_fci_pyscf, fcivec = myfci.kernel()

print(f"Energia Full CI (PySCF):    {e_fci_pyscf:.8f} Hartrees")

# A diferença entre FCI e HF é a energia de correlação
e_corr = e_fci_pyscf - myhf.e_tot
print(f"Energia de Correlação:      {e_corr:.8f} Hartrees")

<class 'pyscf.fci.FCI.<locals>.CISolver'> does not have attributes  nthreads



WARN: Not enough memory for FCI solver. The minimal requirement is 10403 MB



OSError: [Errno 28] Can't write data (file write failed: time = Thu Oct 23 20:05:02 2025
, filename = '/tmp/6frno5pq', file descriptor = 58, errno = 28, error message = 'No space left on device', buf = 0x2b088d9e3810, total write size = 746657504, bytes this sub-write = 746657504, bytes actually written = 18446744073709551615, offset = 0)

Valor experimental: 

Metano,CH₄,-40.526

In [12]:
import pyscf
from pyscf import gto, scf, fci, cc
import dask
from dask_jobqueue import SLURMCluster  # Mude para PBSCluster, SGECluster, etc. se necessário
from dask.distributed import Client
import time
import os

# -----------------------------------------------------------------
# PASSO 1: A FUNÇÃO DE CÁLCULO
# Esta função será serializada e enviada para o HPC.
# Ela deve ser autossuficiente.
# -----------------------------------------------------------------

def run_pyscf_fci(atom_string, basis, spin):
    """
    Executa um cálculo SCF (HF) seguido de Full CI no HPC.
    Recebe strings para construir a molécula remotamente.
    """
    # Importações devem estar DENTRO da função
    from pyscf import gto, scf, fci
    
    # 1. Construir a molécula NO WORKER (no HPC)
    mol = gto.M(
        atom=atom_string,
        basis=basis,
        spin=spin,
        verbose=0 # Importante para não poluir os logs
    )
    mol.build()
    
    # 2. Executar Hartree-Fock
    myhf = scf.RHF(mol).run()
    
    # 3. Executar Full CI (A PARTE PESADA)
    myfci = fci.FCI(myhf)
    e_fci, _ = myfci.kernel()
    
    return e_fci


In [13]:
# -----------------------------------------------------------------
# PASSO 2: DEFINIR OS CÁLCULOS
# -----------------------------------------------------------------

# Molécula leve (para rodar local)
h2_atom = 'H 0 0 0; H 0 0 0.74'
h2_basis = 'sto-3g'
h2_spin = 0

# Molécula PESADA (para rodar no HPC)
# N2 @ 6-31G -> 14 elétrons, 14 orbitais
# Tamanho do FCI: C(14, 7) * C(14, 7) = 3432 * 3432 = 11.7 milhões de termos
# Impraticável em um laptop, fácil para um nó de HPC.
ch4_atom =[
        ['C', ( 0.0,  0.0,  0.0)],
        ['H', (  1.087 / np.sqrt(3),    1.087 / np.sqrt(3),    1.087 / np.sqrt(3) )],
        ['H', (  1.087 / np.sqrt(3),   -1.087 / np.sqrt(3),   -1.087 / np.sqrt(3) )],
        ['H', ( -1.087 / np.sqrt(3),    1.087 / np.sqrt(3),   -1.087 / np.sqrt(3) )],
        ['H', ( -1.087 / np.sqrt(3),   -1.087 / np.sqrt(3),    1.087 / np.sqrt(3) )]
    ]
ch4_basis = '6-31G'
ch4_spin=0,          # Singlete (10 elétrons)

In [14]:
# -----------------------------------------------------------------
# PASSO 3: CÁLCULO LOCAL (TRANSPARENTE)
# -----------------------------------------------------------------
print("--- Iniciando Cálculo LOCAL (H2) ---")
start_local = time.time()

# Simplesmente chamamos a função normalmente
e_local_h2 = run_pyscf_fci(h2_atom, h2_basis, h2_spin)

end_local = time.time()
print(f"Energia Local H2 (FCI/{h2_basis}): {e_local_h2:.8f} Ha")
print(f"Tempo local: {end_local - start_local:.2f} s\n")

--- Iniciando Cálculo LOCAL (H2) ---
Energia Local H2 (FCI/sto-3g): -1.13728383 Ha
Tempo local: 0.04 s



In [15]:
# -----------------------------------------------------------------
# PASSO 4: CONFIGURAR O CLUSTER NO HPC
# -----------------------------------------------------------------
print("--- Configurando Conexão com o Cluster HPC ---")

# (Baseado na configuração do seu HPC)
cluster = SLURMCluster(
    queue='compute',          # Nome da fila/partição do Slurm (ex: 'batch', 'shared')
    project='fci-ch4',# O nome do seu projeto/conta para cobrança
    cores=24,                 # Quantos cores CADA job (worker) deve pedir
    memory='100GB',           # Quanta RAM CADA job (worker) deve pedir
    walltime='01:00:00',      # Tempo máximo do job (HH:MM:SS)
    
    # Configuração de rede (CRUCIAL)
    # Tente 'ib0' (InfiniBand) ou 'eth0' (Ethernet)
    # Peça ao admin do seu HPC qual interface usar
    interface='ib0',
    
    # Onde salvar os logs dos workers no HPC
    log_directory=f'/home/nara/dask-logs',
    
    # Comandos para carregar seu ambiente Python no HPC
    # (Ex: carregar módulos, ativar conda)
    job_extra_directives=[
        'module load python/3.10',
        'source /seu/path/to/venv/bin/activate'
    ]
)

--- Configurando Conexão com o Cluster HPC ---


OSError: [Errno 45] Operation not supported: '/home/nara'

In [17]:

# -----------------------------------------------------------------
# PASSO 5: CONECTAR E SUBMETER O CÁLCULO PESADO
# -----------------------------------------------------------------

scheduler_address = 'tcp://127.0.0.1:8786'

# Conectar seu notebook ao agendador do Dask no HPC
# (Pode ser necessário configurar um Túnel SSH, veja a seção de Segurança)
client = Client(scheduler_address)

# Pedir ao HPC para iniciar 1 "job" (worker)
# (Isso vai submeter 1 script sbatch)
print("Solicitando 1 nó de trabalho do HPC...")
cluster.scale(jobs=1) 

print(f"Cluster iniciado! Dashboard: {client.dashboard_link}")
print("Aguardando o job ser alocado pelo Slurm... (Isso pode levar minutos)")

# Espera até que o worker esteja pronto
client.wait_for_workers(1) 

print("\n--- Iniciando Cálculo REMOTO (N2) no HPC ---")
start_remote = time.time()

# 1. Submeter o cálculo pesado. Isso retorna um "Future" (uma promessa)
future_n2 = client.submit(run_pyscf_fci, n2_atom, n2_basis, n2_spin)

# 2. Pegar o resultado.
# O .result() é bloqueante: seu notebook vai esperar aqui
# até que o HPC termine o cálculo e envie o resultado.
e_remote_n2 = future_n2.result() 

end_remote = time.time()
print(f"Energia Remota N2 (FCI/{n2_basis}): {e_remote_n2:.8f} Ha")
print(f"Tempo remoto (incluindo espera): {end_remote - start_remote:.2f} s")

CommClosedError: in <TCP (closed)  local=tcp://127.0.0.1:59977 remote=tcp://127.0.0.1:8786>: ConnectionResetError: [Errno 54] Connection reset by peer

In [ ]:
# -----------------------------------------------------------------
# PASSO 6: DESLIGAR
# -----------------------------------------------------------------
print("\n--- Desligando ---")
client.close()
cluster.close()
print("Cálculos concluídos. Cluster desligado.")

In [1]:
from mpi4pyscf import scf    # substitui pyscf.scf
from mpi4pyscf import df     # (opcional) p/ density-fitting paralela
from pyscf import gto

mol = gto.M(atom="H 0 0 0; H 0 0 0.74", basis="cc-pVTZ", unit="Bohr")

# SCF paralelo (cada rank faz parte do trabalho)
mf = scf.RHF(mol)
mf.max_memory = 48_000
E = mf.kernel()
if mf.mpi_rank == 0:
    print("E(SCF) =", E)

ModuleNotFoundError: No module named 'mpi4pyscf'